### an overly broad introduction to a selection of data science and machine learning topics, including&mdash;but not limited to&mdash;terminology; basic pipelines; and quantifying uncertainty, by someone who has worked on some but not all of this before


##### *this interactive slideshow was made with RISE*

this notebook was written by wil ward

- **some terminology**
- **more terminology**
- **types of data**
- **preparing data**
- **regression**
- **non-linear regression**
- **unsupervised learning**
- **key points**
- **anything else**

In [ ]:
# some imports
import numpy as np
import os, sys

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn

seaborn.set_theme(
    context="talk",
    rc={"lines.linewidth": 2},
    palette='colorblind',
    font_scale=1)

import pandas as pd

from PIL import Image

import warnings
warnings.filterwarnings("ignore")

## some terminology

**data**

a collection of information, typically numeric but possibly comprising words or images

data are not their interpretation

**feature**

a property of the data

**model**

an abstract representation of a system used to make sense of data and its features

**decision**

the execution of a specific rule or choice given an input

**prediction**

a guess at what the features might show given specific conditions

**inference / training**

making a model represent the data and its features

**objective function / loss**

how close a model outputs are to the data features

**learning algorithm**

the specific _choice_ of model(s) and training used to represent the data

## what is ...



### data science

the study of acquiring knowledge from data, including acquisition; curation; analysis and visualisation

### artificial intelligence

a (computer) system that can make decisions and predictions

### machine learning

the automatic development of _rules_ without explicit intervention

### big data

data that either
- consists of a huge number of data
- has lots of features

<center><img width=1200px src="https://wilocw.github.io/files/resource/global/datasci.png"></img></center>

## more terminology

#### data

$X$ inputs (sometimes $t$ if time-series)

$y$ outputs/observations

#### model

$\theta$ parameters (these are learned)

$\alpha$ hyperparameters (these are fixed)

$f$ learnable function, e.g. $f(X,\theta;\alpha)$

$\varepsilon$ (observation) noise, typically Gaussian

#### model

$\mathcal{L}$ objective function

$\pi$ observation model / likelihood

$z$ intermediate (latent) variable(s)

example:
$$
    z = f(X,\theta)\qquad\qquad
    y = \pi(z)
$$
where $\pi(z) = z + \varepsilon$, and $\varepsilon \sim \mathrm{N}(0,\sigma^2)$

## types of data

<div class='container'>
    
<div class='col' style='text-align: left'>
    <h3>continuous</h3>
    <br />
    time-series [$y = f(t)$]<br />
    spatial data [$y = f(\mathbf{x})$]<br />
    spatio-temporal [$y = f(\mathbf{x},t)$]
</div>
       
<div class='col' style='text-align: left'>
    <h3>discrete</h3><br />
    classification<br />
    count data [$y_k = \lambda(x_k)$]<br />
    images [$y_k = f(I_k)$]<br />
    graphs / networks [$y = f(G(X),W)$]
</div>
    
</div>

continuous functions are easier to model

pretty much nothing is ever _measured_ continuously

## preparing data

data quality has direct impact on analysis and predictions

curation and storage important factors

considerations include privacy, access, visualisation

few established best practices

outlier detection, dimensionality reduction, summary statistics

data readiness a contemporary issue:<br/> [The DELVE Initiative (2020), Data Readiness: Lessons from an Emergency. DELVE Report No. 7. Published 24 November 2020](https://rs-delve.github.io/reports/2020/11/24/data-readiness-lessons-from-an-emergency.html#key-points)

DELVE = Data Evaluation and Learning for Viral Epidemics

A multi-disciplinary group, convened by the Royal Society, to support a data-driven approach to learning from the different approaches countries are taking to managing the covid-19 pandemic. 

The report builds on the experience of the Royal Society’s DELVE group in providing rapid turnaround, data-driven answers to policy questions, and the challenges in access to data that arose during this work. It sets out the challenges we found that were common to these data sharing activities, focussing on the use of new data modalities, arising from, for example mobile phone usage or electronic payments data, rather than the classical challenges of collecting surveillance data. It then considers recommendations for addressing these challenges and creating an environment that supports the safe and rapid use of a range of different types of data in policymaking.

### example

In [ ]:
data = pd.read_csv('https://wilocw.github.io/files/resource/global/2019.csv')
data.rename(lambda s: s.replace(' ','_'), axis='columns', inplace=True)

In [ ]:
data

_World Happiness Report, 2019. UN Sustainable Development Solutions Network_.<br/>https://www.kaggle.com/unsdsn/world-happiness/metadata

#### extract features

In [ ]:
features = ['GDP_per_capita', 'Social_support', 'Life_expectancy', 'Freedom', 'Generosity', 'Corruption']

X = data.loc[:,features].values

y = data.loc[:,['Score']].values

In [ ]:
seaborn.pairplot(data, vars=features, hue='Continent', diag_kind='kde', plot_kws={'s':100});

#### data standardisation

normalise features to give consistent statistical properties

$$ \frac{X - \mathrm{mean}(X)}{\mathrm{std}(X)} $$

In [ ]:
X = (X - np.mean(X, axis=0)) / (np.std(X, axis=0))

print('standardised mean: %2.1f \nstandardised std: %2.1f' % (np.mean(X),np.std(X)))

#### dimensionality reduction

reduce number of relative features by taking into account internal correlations

use principle component analysis (PCA)

In [ ]:
from sklearn.decomposition import PCA

def plot(pc):
    ''' this is a utility function to plot one or two principle components '''
    if pc.shape[1] == 1:
        df = pd.DataFrame({'pc1': pc.ravel(), 'Continent': data.loc[:,['Continent']].values.ravel(), '0': np.zeros_like(pc.ravel())})

        p = seaborn.FacetGrid(df, aspect=8, height=2)
        p.map_dataframe(seaborn.scatterplot, x='pc1', y='0', hue='Continent', s=200)
        p.add_legend()
        p.set_axis_labels('pc1','')
        p.set(yticks=[]);
    else:
        df = pd.DataFrame({'pc1': pc[:,0], 'pc2': pc[:,1], 'Continent': data.loc[:,['Continent']].values.ravel()})

        p = seaborn.FacetGrid(df, height=7)
        p.map_dataframe(seaborn.scatterplot, x='pc1', y='pc2', hue='Continent', s=200)
        p.set_axis_labels('pc1','pc2')
        p.add_legend()

In [ ]:
z = PCA(1).fit_transform(X)
plot(z)

In [ ]:
z = PCA(2).fit_transform(X)
plot(z)

dimensionality reduction finds a _latent_ representation to represent data in lower number of features

an example of unsupervised learning

no free lunch

other examples: _autoencoders_, _GP-LVM_, _neural networks_

## regression

In [ ]:
### this is an example of linear regression on one feature against the "happiness" score

def regression_example():
    ''' this function will create an interactive regression tool'''
    
    # reinitialise data 
    X, y = data.loc[:,features].values, data.loc[:,['Score']].values
    # standardise
    X = (X - np.mean(X, axis=0)) / (np.std(X, axis=0))
    y = (y - np.mean(y)) / np.std(y)

    # dummy X for plotting a straight line (only need end points)
    dummy_x = np.array([[-2.5],[2.2]])

    def reset_ax1(x,y):
        ''' this will reset the right hand axis (interactive) '''
        axs[1].clear()

        axs[1].plot([-2, x], [y,y], 'r-')
        axs[1].plot([x,x], [-2, y], 'r-')

        axs[1].text(x - 1, y+0.1,'%5.4f, %5.4f' % (x, y))

        axs[1].set_xlim([-2,2])
        axs[1].set_ylim([-2,2])
        
        axs[1].set_xlabel(r'$\theta_0$')
        axs[1].set_ylabel(r'$\theta_1$')

    def onpick(event):
        ''' this will update (and plot) the fit with the selected parameters '''
        if event.x < 450: 
            return
        # reset left hand plot
        axs[0].clear()
        plot_gdpexp(axs[0])
        # plot f(x) = t0 + t1*x
        dummy_y = event.ydata*dummy_x + event.xdata
        axs[0].plot(dummy_x, dummy_y, 'm-', label=r'$\theta_0 + \theta_1x$')
        # misc. details
        axs[0].legend(loc='lower right')
        axs[0].set_title(r'$\mathcal{L} = %5.3f$'%r2(event.xdata,event.ydata))
        reset_ax1(event.xdata, event.ydata)

    def plot_gdpexp(ax):
        ''' this just plots the training data '''
        ax.plot(X[:,0], y,'.', label=None)
        ax.set_xlim([-2.5, 2.2])
        ax.set_ylim([-2.54, 2.54])

        axs[0].set_xlabel(features[0])
        axs[0].set_ylabel('Score')

    def r2(t0,t1):
        ''' square error '''
        f = X[:,0]*t1 + t0
        return np.sum((y.ravel()-f)**2)
    
    # create a plot
    fig, axs = plt.subplots(1,2, figsize=(14, 6))
    # plot the data
    plot_gdpexp(axs[0])
    # plot the default straight like f(x) = 0 + 0x
    axs[0].plot(dummy_x, dummy_x*0 + 0, 'm-', label=r'$\theta_0 + \theta_1x$')
    axs[0].legend(loc='lower right')
    axs[0].set_title(r'$\mathcal{L} = %5.3f$'%r2(0,0))
    # initialise (reset) the right hand plot
    reset_ax1(0,0)
    # add callback to mouse click on axes
    fig.canvas.mpl_connect("button_press_event", onpick)
    plt.tight_layout()
    

### linear regression

$$ f(x, \theta) =  \theta_0 + \theta_1x $$

minimise $$\mathcal{L} = \sum (y - f(X, \theta))^2$$

In [ ]:
%matplotlib notebook
regression_example()

<center><h2>machine learning is optimisation of statistical metrics</h2></center>

$$
\text{find } f^* \in \mathcal{F} \text{ such that}\qquad\qquad\qquad
$$

$$
    f^* = \arg\min_{f\in\mathcal{F}} \mathcal{L}(y,f(X))
$$

<center><h2>machine learning is optimisation of statistical metrics</h2></center>


$$
\text{find } \theta^* \in \Theta \text{ such that}\qquad\qquad\qquad
$$

$$
    \theta^* = \arg\min_{\theta\in\Theta} \mathcal{L}(y,f(X,\theta))
$$

### training a regressor

In [ ]:
from scipy.optimize import minimize
from matplotlib.colors import LogNorm

def trained_regression_example():
    ''' this will automatically fit a curve and plot the learning path '''
    
    def callback(itr):
        # save the optimiser step for plotting later
        ts.append(itr)
        
    # reinitialise data
    X, y = data.loc[:,features].values, data.loc[:,['Score']].values
    # standardise
    X = (X - np.mean(X, axis=0)) / (np.std(X, axis=0))
    y = (y - np.mean(y)) / np.std(y)
    
    # dummy X for plotting a straight line (only need end points)
    dummy_x = np.array([[-2.5],[2.2]])
    
    # resolution for plotting loss fcn (res x res)
    res = 100
    t0,t1 = np.meshgrid(np.linspace(-2,2,res),np.linspace(-2,2,res))

    def r2(t):
        ''' square error '''
        f = X[:,0]*t[1] + t[0]
        return np.sum((y.ravel()-f)**2)

    # calculate the full loss surface (this is easy for 2 parameters, but non-trivial usually)
    L_ = np.array([[r2([t0[i,j],t1[i,j]]) for j in range(res)] for i in range(res)])
   
    # initial estimate (arbitrary)
    t_0 = np.array([1,-1])
    # save optimiser guesses in ts
    ts = [t_0]
    
    # bounded minimisation of r2(theta)
    opt = minimize(r2, t_0, method='L-BFGS-B', bounds=((-2,2),(-2,2)), callback=callback)

    def plot_gdpexp(ax):
        ''' this just plots the training data '''
        ax.plot(X[:,0], y,'.', label=None)
        ax.set_xlim([-2.5, 2.2])
        ax.set_ylim([-2.54, 2.54])

        axs[0].set_xlabel(features[0])
        axs[0].set_ylabel('Score')
    
    # create a plot
    fig, axs = plt.subplots(1,2, figsize=(14,6))
    # plot the data
    plot_gdpexp(axs[0])
    
    # for each optimiser step, plot the fit
    for t in ts[::-1]:
        axs[0].plot(dummy_x, dummy_x*t[1] + t[0], 'k-', alpha=0.1)
    # plot final guess
    axs[0].plot(dummy_x, dummy_x*ts[-1][1] + ts[-1][0], 'm-')
    
    # on r.h. plot, show the loss surface
    axs[1].pcolor(t0, t1, L_, norm=LogNorm(vmin=L_.min(), vmax=L_.max()))
    # ... and the optimiser search path
    axs[1].plot([t[0] for t in ts], [t[1] for t in ts], 'ro-')
    
    # misc. details
    axs[1].set_xlim([-2,2])
    axs[1].set_ylim([-2,2])
    axs[1].set_xlabel(r'$\theta_0$')
    axs[1].set_ylabel(r'$\theta_1$')
    
    plt.tight_layout()


In [ ]:
%matplotlib inline
trained_regression_example()

### a Bayesian example

prior: $p(\theta) = \mathrm{N}(\theta\,|\,\mathbf{0},\mathbf{I})$

likelihood: $p(y\,|\,X,\theta) = \mathrm{N}(y\,|\, \theta_0 + \theta_1X, \sigma^2)$

we are interested in an estimate of $\theta$ conditioned on the data. the posterior: $p(\theta\,|\,X,y)$

### prior knowledge

In [ ]:
from scipy.stats import multivariate_normal as mvn

def plot_prior():
    ''' this will plot fits based on samples from the prior of theta '''
    # reinitialise the data
    X, y = data.loc[:,features].values, data.loc[:,['Score']].values
    # standardise
    X = (X - np.mean(X, axis=0)) / (np.std(X, axis=0))
    y = (y - np.mean(y)) / np.std(y)

    # prior p(theta) = N(0, I) [mvn = multivariate normal]
    p_t = mvn(np.zeros(2,), np.eye(2))
    
    # dummy X for plotting a straight line (only need end points)
    dummy_x = np.array([[-2.5],[2.2]])
    
    # resolution for plotting prior (res x res)
    res = 100
    t0,t1 = np.meshgrid(np.linspace(-2,2,res),np.linspace(-2,2,res))
    
    def prior(ax):
        ''' contour plot of the prior p_t '''
        ax.contour(t0,t1, np.array([[p_t.pdf([t0[i,j],t1[i,j]]) for j in range(res)] for i in range(res)]),'k-', levels=5)
        
    def plot_gdpexp(ax):
        ''' this just plots the training data '''
        ax.plot(X[:,0], y,'.', label=None)
        ax.set_xlim([-2.5, 2.2])
        ax.set_ylim([-2.54, 2.54])

        axs[0].set_xlabel(features[0])
        axs[0].set_ylabel('Score')
    
    # create a plot
    fig, axs = plt.subplots(1,2, figsize=(14,6))
    # plot the data
    plot_gdpexp(axs[0])
    
    # draw 100 random variable samples (rvs) from the prior p(theta)
    ts = p_t.rvs(100)
    # for each sample of theta, plot the corresponding linear fit
    for i in range(100):
        axs[0].plot(dummy_x, dummy_x*ts[i,:][1] + ts[i,:][0], 'k-', alpha=0.2,lw=0.5)
    
    # plot the prior on the r.h. plot
    prior(axs[1])
    # plot the samples
    axs[1].plot(ts[:,0],ts[:,1],'k.',ms=5)
    
    # misc. details
    axs[1].set_title(r'$p(\theta)$')
    axs[1].set_xlim([-2,2])
    axs[1].set_ylim([-2,2])
    axs[1].set_xlabel(r'$\theta_0$')
    axs[1].set_ylabel(r'$\theta_1$')
    
    plt.tight_layout()

In [ ]:
%matplotlib inline
plot_prior()

In [ ]:
def plot_posterior(s2=1.):
    ''' this will plot fits based on posterior p(theta|X,y) based on observation noise s2 '''
    # reinitialise the data
    X, y = data.loc[:,features].values, data.loc[:,['Score']].values
    # standardise
    X = (X - np.mean(X, axis=0)) / (np.std(X, axis=0))
    y = (y - np.mean(y)) / np.std(y)
    
    # dummy X for plotting a straight line (only need end points)
    dummy_x = np.array([[-2.5],[2.2]])
    
    ## we add a dummy variable of 1s so we can do some linear algebra
    x_ = np.vstack((np.ones_like(X[:,0]), X[:,0])).T
    
    # resolution for plotting prior (res x res)
    res=100
    t0,t1 = np.meshgrid(np.linspace(-2,2,res),np.linspace(-2,2,res))
    
    # inferred posterior (this is a closed form calculation [not really ML])
    def p_t_given_xy():
        ''' the posterior p(theta|X,y) based on obs. noise s2 '''
        Sig = np.linalg.inv(np.eye(2) + (x_.T@x_)/s2)
        mu = Sig@np.sum(y*x_, axis=0)/s2
        return mvn(mu,Sig) # N(mu, Sig)

    def posterior(ax):
        ''' contour plot of the prior p_t '''
        ax.contour(t0,t1, np.array([[p_t_given_xy().pdf([t0[i,j],t1[i,j]]) for j in range(res)] for i in range(res)]),'k-', levels=5)
        
    # this is the negative log likelihood of our estimate (the lower the better !)
    negloglik = lambda s: -mvn(np.zeros_like(y).ravel(), (x_@x_.T) + s*np.eye(y.shape[0])).logpdf(y.ravel())
    
    def plot_gdpexp(ax):
        ''' this just plots the training data '''
        ax.plot(X[:,0], y,'.', label=None)
        ax.set_xlim([-2.5, 2.2])
        ax.set_ylim([-2.54, 2.54])

        axs[0].set_xlabel(features[0])
        axs[0].set_ylabel('Score')
        axs[0].set_title(r'$\sigma^2$=%5.4f  ; $\mathcal{L}(\sigma^2)$ = %5.4f' % (s2,negloglik(s2)))
    
    # create a plot
    fig, axs = plt.subplots(1,2, figsize=(14,6))
    # plot the data
    plot_gdpexp(axs[0])
    
    # draw 100 random variable samples (rvs) from the posterior p(theta | X,y)
    ts = p_t_given_xy().rvs(100)
    # for each sample of theta, plot the corresponding linear fit
    for i in range(100):
        axs[0].plot(dummy_x, dummy_x*ts[i,:][1] + ts[i,:][0], 'k-', alpha=0.2,lw=0.5)

    # plot the posterior in the r.h. plot
    posterior(axs[1])
    # plot the samples of theta
    axs[1].plot(ts[:,0],ts[:,1],'k.',ms=5)
    
    # misc. details
    axs[1].set_title(r'$p(\theta\,|\,X,y)$')
    axs[1].set_xlim([-2,2])
    axs[1].set_ylim([-2,2])
    axs[1].set_xlabel(r'$\theta_0$')
    axs[1].set_ylabel(r'$\theta_1$')
    
    plt.tight_layout()

In [ ]:
%matplotlib inline
plot_posterior(s2 = 1.)

the next lines of code optimise the observation noise to jointly learn all parameters !

In [ ]:
x_ = np.vstack((np.ones_like(X[:,0]), X[:,0])).T
# this is the negative log likelihood of our estimate (the lower the better !)
negloglik = lambda s: -mvn(np.zeros_like(y).ravel(), (x_@x_.T) + s*np.eye(y.shape[0])).logpdf(y.ravel())

In [ ]:
opt = minimize(negloglik, [1.], method='L-BFGS-B', bounds=((1e-5, 10.),))

plot_posterior(opt.x)

In [ ]:
def plot_linear_errorbars(s2=1.):
    ''' this will plot the uncertainty (95% CI) on posterior p(theta|X,y) based on observation noise s2 '''
    
    # reinitialise the data
    X, y = data.loc[:,features].values, data.loc[:,['Score']].values
    # standardise
    X = (X - np.mean(X, axis=0)) / (np.std(X, axis=0))
    y = (y - np.mean(y)) / np.std(y)
    
    ## we add a dummy variable of 1s so we can do some linear algebra
    x_ = np.vstack((np.ones_like(X[:,0]), X[:,0])).T
    
    # resolution for plotting prior (res x res)
    res=100
    t0,t1 = np.meshgrid(np.linspace(-2,2,res),np.linspace(-2,2,res))
    
    # inferred posterior (this is a closed form calculation [not really ML])
    def p_t_given_xy():
        ''' the posterior p(theta|X,y) based on obs. noise s2 '''
        Sig = np.linalg.inv(np.eye(2) + (x_.T@x_)/s2)
        mu = Sig@np.sum(y*x_, axis=0)/s2
        return mvn(mu,Sig) # N(mu, Sig)
    
    def plot_gdpexp(ax):
        ''' this just plots the training data '''
        ax.plot(X[:,0], y,'.', label=None)
        ax.set_xlim([-2.5, 2.2])
        ax.set_ylim([-2.54, 2.54])

        ax.set_xlabel(features[0])
        ax.set_ylabel('Score')
        ax.set_title(r'$\sigma^2$=%5.4f  ; $\mathcal{L}(\sigma^2)$ = %5.4f' % (s2,negloglik(s2)))

    # create a plot
    fig, ax = plt.subplots(1,1, figsize=(7,6))
    
    # we take lots of samples to do Monte Carlo estimates of uncertainty
    ts = p_t_given_xy().rvs(1000)    
    # resolution of fit (we need this to get estimates of uncertainty all the way across the fit)
    ln = 200
    # dummy X for plotting a straight line (need more points)
    dummy_x = np.linspace(-2.5, 2.2, ln)
    # calculate fits for each sample of theta
    dummy_y = np.vstack([dummy_x*ts[i,:][1] + ts[i,:][0] for i in range(1000)])
    
    # calculate 95% CI (2.5, 97.5 quantiles) of fits
    qnt = np.quantile(dummy_y, q=(0.025, 0.975), axis=0)
    # plot 95% CI with observation noise
    ax.fill_between(dummy_x, qnt[0,:] - 1.96*s2, qnt[1,:] + 1.96*s2, alpha=0.25, label='observation 95% CI')
    # plot 95% CI without observation noise (effects of theta only)
    ax.fill_between(dummy_x, qnt[0,:], qnt[1,:], alpha=1., label='latent 95% CI')
    
    # calculate the mean fit
    mu = p_t_given_xy().mean
    # plot the mean fit
    ax.plot(dummy_x, dummy_x*mu[1] + mu[0], 'k-', lw=2, label=r'$\mathbf{E}[y\,|\theta]$')

    # plot the data
    plot_gdpexp(ax)
    
    # add a legend
    ax.legend()
    plt.tight_layout()

In [ ]:
plot_linear_errorbars(opt.x)

### generalised linear models

$$ f(X,\theta) = \theta_0 + \theta_1X_1 + \theta_2X_2 + \ldots$$

$$\theta \sim p(\theta)$$
$$y \sim \pi(f(X,\theta),\alpha)$$

this example is a bit more involved, computationally, and requires using Markov chain Monte Carlo (MCMC) -- skip it if you like

In [ ]:
import pymc3 as pm
from sklearn.preprocessing import StandardScaler


def plot_glm():
    ''' calculate and plot the generalised linear model '''
    
    ## reload and reinitialise all the data (just in case)
    data = pd.read_csv('https://wilocw.github.io/files/resource/global/2019.csv')
    data.rename(lambda s: s.replace(' ','_'), axis='columns', inplace=True)

    data = pd.DataFrame(StandardScaler().fit_transform(data[['Score'] + features]), columns=['Score'] + features)

    # GLM formula "Score ~ GDP_per_capita + Social_support + Life_expectancy + Freedom + Generosity + Corruption"
    formula = 'Score ~' + ''.join([' ' + feature + ' +' for feature in features])[:-2]

    # context for the model
    with pm.Model() as normal_model:

        # the prior for the likelihood
        family = pm.glm.families.Normal()

        # creating the model requires a formula and data (and optionally a family)
        pm.GLM.from_formula(formula, data = data, family = family)

        # perform Markov chain Monte Carlo sampling
        normal_trace = pm.sample(draws=2000, chains = 2, tune = 1000)
        # sample the posterior for features and intercept
        posterior_predictive = pm.sample_posterior_predictive(normal_trace, var_names=features + ['Intercept'])

    # calculate the median value of posterior
    medians = {feature: np.median(posterior_predictive[feature]) for feature in features}
    
    # create a plot
    fig, axs = plt.subplots(2,3, figsize=(14, 12))

    # for each feature, plot the linear fit
    for i, feature in enumerate(features):
        # index for plots (equiv to ind2sub in matlab)
        j,k = i // 3, i % 3
        # get the current feature
        x_ = data[feature]
    #     axs[j,k].plot(x_, data['Score'],'.', label=None) # its actually not great a fit so this is commented out for the talk
        axs[j,k].set_xlabel(feature)
        if k == 0:
            axs[j,k].set_ylabel('Score')
        # For a subset posterior sample (1000 is too many to plot), fix the other features (to median) and create a fit
        for t0,t1 in zip(posterior_predictive['Intercept'][::40],posterior_predictive[feature][::40]):
            f = t0 + t1*x_ + np.sum([data[nfeature]*medians[nfeature] for nfeature in features if nfeature is not feature])
            axs[j,k].plot(x_, t0 + t1*x_, 'k-', alpha=0.2, lw=0.5)

In [ ]:
plot_glm()

### would it be easier to predict trends via PCA ?

In [ ]:
def plot_pc_score(z):  
    ''' this function plots the princple components against y (Score) '''
    # create plots
    _, axs = plt.subplots(1,2,figsize=(14, 6), sharex=True, sharey=True)
    # plot pc1 vs score
    axs[0].plot(z[:,0], y, 'o')
    axs[0].set_xlabel('pc1')
    axs[0].set_ylabel('score')
    # plot pc2 vs score
    axs[1].plot([],[],'o')
    axs[1].plot(z[:,1], y, 'o')
    axs[1].set_xlabel('pc2')
    plt.tight_layout()

In [ ]:
z = PCA(2).fit_transform(X)
plot_pc_score(z)

In [ ]:
## exercise left to the reader ! fit a linear model to the PCA transformation and 

data = pd.read_csv('https://wilocw.github.io/files/resource/global/2019.csv')
data.rename(lambda s: s.replace(' ','_'), axis='columns', inplace=True)

X = StandardScaler().fit_transform(data.loc[:,features].values)
z = PCA(2).fit_transform(X)

# ...

## nonlinear regression

### nonlinear regression

most functions and models aren't linear

(but sometimes it's easier to pretend they are)

a popular method is to linearise data

other methods include polynomial fitting; Gaussian processes; and neural networks

### general regression framework

$ z = f(X)$ ; $y \sim \pi(z)$ ; $f \in \mathcal{F}$

$$
\text{find } f^* \in \mathcal{F} \text{ such that}\qquad\qquad\qquad
$$

$$
    f^* = \arg\min_{f\in\mathcal{F}} \mathcal{L}(y,\pi(z))
$$

choice of $\mathcal{F}$, $\pi$ and $\mathcal{L}$ define model

choice of function space, likelihood (observation model), and objective metric define model

In [ ]:
# set a random seed to get data 
np.random.seed(1234)

# generative function
f = lambda x: (10*np.cos(np.pi*x)/3 + np.sin(np.pi*x*2) + x**3 + 4*x*x)/20

# create a mesh for plotting functions
nX = np.linspace(-2,2, 250)
# random X
X = np.random.choice(nX, (20))[:,None]
# y = f(X) + noise
y = f(X) + 0.1*np.random.randn(20,1)

def plot_nl_data():
    ''' plot the training data '''
    plt.figure(figsize=(14, 5))
    plt.plot(X, y, 'o')
    plt.xlabel(r'$X$')
    plt.ylabel(r'$y$')

In [ ]:
%matplotlib inline
plot_nl_data()

### example

polynomial regression

$$f(X,\theta) = \theta_0 + \theta_1X + \theta_2X^2 + \ldots$$

$$\pi(z) = z + \epsilon$$

$$ \mathcal{L} = \sum \epsilon^2 $$

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

def fit_polys():
    ''' fit polynomials with different orders '''
    
    # create a plot
    _, axs = plt.subplots(1,3,figsize=(16, 4), sharey=True)
    # for degree = 1, 4, and 10
    for i,d in enumerate([1,4,10]):
        # build the training pipeline in sklearn
        pipeline = Pipeline([("polynomial_features", PolynomialFeatures(degree=d, include_bias=True)),
                             ("linear_regression", LinearRegression())])
        # fit the data
        pipeline.fit(X,y)
        # plot the training data
        axs[i].plot(X, y, 'o')
        # plot the the trained curve
        axs[i].plot(nX, pipeline.predict(nX[:,None]),'-', label='num. params '+str(d+1))
        # misc. details
        axs[i].set_xlabel('X')
        axs[i].legend(loc='upper left')
        axs[i].set_ylim((-2.1,2.1))
    axs[0].set_ylabel('y')
    plt.tight_layout()

In [ ]:
%matplotlib inline
fit_polys()

parameters $\theta$ are polynomial coefficients

hyperparameters $\alpha$ is polynomial order (num. params - 1)

### example

Gaussian processes

$$f(X) \sim \mathrm{GP}(m(X), k(X,X'))$$

$$\pi(z) = \mathcal{N}(z, \sigma^2)$$

$$ \mathcal{L} = -\log p(y\,|\,X,\alpha) $$

In [ ]:
# sklearn is not a great library for GPs but it's easy enough to use here
import sklearn.gaussian_process as gp 
from sklearn.gaussian_process.kernels import Matern,ConstantKernel, DotProduct, WhiteKernel

def fit_gps():
    ''' we will fit some different Gaussian processes to the data '''
    
    # create a plot
    _, axs = plt.subplots(3,3,figsize=(16, 13), sharey=True, sharex=True)
    
    # different covariance assumptions
    kernels = [
        DotProduct(sigma_0=0.1) , # this is equivalent to Bayesian linear regression !
        ConstantKernel() + Matern(length_scale=0.01, nu=1/2), # this assumes similarity (covariance) decays exponentially
        ConstantKernel() + Matern(length_scale=10., nu=5/2)   # more smooth assumption of latent functions
    ]
    # for each kernel
    for i,k in enumerate(kernels):
        # fit a GP model with kernel k (assumes fixed [known] observation noise)
        gpr = gp.GaussianProcessRegressor(kernel=k, alpha=0.01, n_restarts_optimizer=10)
        gpr.fit(X,y)
        
        # we extract a mean and covariance for our latent GP
        m, S = gpr.predict(nX[:,None], return_cov=True)
        # ... from which we can construct a posterior of f(X)
        p_f = mvn(m.ravel(),S+1e-7*np.eye(len(m)))
        
        # plot the data and the mean fit
        axs[0,i].plot(X, y, 'o')
        axs[0,i].plot(nX, m, '-', lw=2, label=r'$\mathbb{E}[f]$')
        axs[0,i].set_title(['Linear', 'Exponential', 'Matern 5/2'][i])
        axs[0,i].legend(loc='upper left')
        
        # plot samples from the posterior (most representative of the function estimates)
        axs[1,i].plot(X, y, 'o')
        axs[1,i].plot(nX, p_f.rvs(1).T, 'k-', lw=0.5, alpha=0.5, label=r'samples of $f$')
        axs[1,i].plot(nX, p_f.rvs(9).T,'k-', lw=0.5, alpha=0.5, label=None)
        axs[1,i].legend(loc='upper left')
        
        # plot the 95% CI of the latent and observation models
        axs[2,i].fill_between(nX, m.ravel() - 1.96*(np.sqrt(np.diag(S+0.01))), m.ravel() + 1.96*(np.sqrt(np.diag(S+0.01))), alpha=0.25, label='observation 95% CI')
        axs[2,i].fill_between(nX, m.ravel() - 1.96*np.sqrt(np.diag(S)), m.ravel() + 1.96*np.sqrt(np.diag(S)), alpha=1., label='latent 95% CI')
        axs[2,i].plot(X, y, 'o')
        axs[2,i].legend()
    
    # misc. details
    for i in range(3):
        axs[i,0].set_ylabel('y')
        axs[2,i].set_xlabel('X')
    plt.tight_layout()

In [ ]:
%matplotlib inline
fit_gps()

### example

neural network

$$f(X, \theta) = \sigma_0(\sum\theta_{w_1}\sigma_{1}(\sum\theta_{w_2}\sigma_{2}(\ldots) + \theta_{b_2}) + \theta_{b_1})$$

$$\pi(z) = \theta_{w_0}z + \theta_{b_0} $$

$$ \mathcal{L} = \sum(y - \pi(z))^2$$

<center><img width=1200px src="https://wilocw.github.io/files/resource/global/nn_example.png"></img></center>

In [ ]:
# again, sklearn is not the ideal library but it serves a purpose here
from sklearn import neural_network as nn

def fit_nn():
    ''' fit a multilayer perceptrons (neural networks) of different sizes and plot '''
    
    # this is utility function that will create the neural network for a given layer architecture
    mlpr = lambda layers: nn.MLPRegressor(hidden_layer_sizes=layers, activation='logistic', solver='lbfgs')

    # create plot
    _, axs = plt.subplots(1,3,figsize=(16, 4), sharey=True, sharex=True)

    # different layer architectures (see sketches above !)
    layers = [(1,), (5,), (100,100)]
    # for each one, create and fit 
    for i,layer in enumerate(layers):
        # create neural network
        mlpr_ = mlpr(layer)
        # train the nn
        mlpr_.fit(X, y)
        # plot the training data
        axs[i].plot(X, y, 'o')
        # plot the predicted function from the trained model
        axs[i].plot(nX, mlpr_.predict(nX[:,None]), label='num. params ' + str(np.sum([len(coef) for coef in mlpr_.coefs_])))
        # misc. details
        axs[i].legend(loc='upper left')
        axs[i].set_title('hidden layer size: ' + str(layer))
        axs[i].set_xlabel('X')
    axs[0].set_ylabel('y')
    plt.tight_layout()

In [ ]:
%matplotlib inline
fit_nn()

parameters $\theta$ are neural network weights and biases

hyperparameters $\alpha$ is number and dimensionality of layers

### considerations for regression

what are the properties of your data ? do you have lots of it ? does it have a lot of features ? do you have high dimensional output ?

do you know anything about the trends already ? is it (mostly) linear ? is it smooth ? 

do you care about uncertainty [n.b. you should !] ?

do easy models work fine ?

### what about classification ?

it's just regression !

most regression methods (sometimes _implicitly_) assume Gaussian noise in the error

it's easier that way

classification is just an example where the observation model $\pi$ is not Gaussian

examples include logistic regression; categorical likelihoods (for Bayesian models) and neural networks with a softmax layer

In [ ]:
# this is an example of classification
## we assume a latent function (z = f(X)) is a GP with a categorical likelihood (y = pi(z))

global plot_probs # this is a bit of trickery to save classifer state outside the demo function

def plot_classifier():
    ''' this will train a GP classifier and plot the decision boundaries '''
    
    # reinitialise the data 
    data = pd.read_csv('https://wilocw.github.io/files/resource/global/2019.csv')
    data.rename(lambda s: s.replace(' ','_'), axis='columns', inplace=True)

    # standardise the data (this is equiv to do it manually it's just easier this way)
    X = StandardScaler().fit_transform(data.loc[:,features].values)
    
    # we're going to classify the 2 principle components from earlier in the talk 
    z = PCA(2).fit_transform(X)

    # convert text labels into integers (1 to 6)
    y = np.array([{'Africa':1, 'Asia':2, 'Europe':3, 'North America':4,'Oceania':5,'South America':6}[cont] for cont in data.loc[:,'Continent']])

    # plotting limits
    lims = ((-4, 5), (-3, 4))

    # classifier model (Matern 5/2 covariance)
    gpc = gp.GaussianProcessClassifier(kernel=Matern(nu=5/2),n_restarts_optimizer=10).fit(z,y)

    # create a mesh to plot 2-D classification surface on
    xg,yg = np.meshgrid(np.linspace(*lims[0],50),np.linspace(*lims[1],50))
    nX = np.vstack((xg.ravel(), yg.ravel())).T

    # class labels for nX
    C = gpc.predict(nX)

    # create a plot
    fig,axs = plt.subplots(1,2,figsize=(16,5), sharex=True, sharey=False)
    # plot the training data on the left hand plot
    for i in range(1,7):
        z_ = z[np.where(y==i),:][0,...]
        axs[0].plot(z_[:,0],z_[:,1],'o',label=['Africa','Asia','Europe','North America','Oceania','South America'][i-1])

    # plot the classifier C
    cax = axs[1].contourf(xg, yg, C.reshape(50,50), levels=np.array([1,2,3,4,5,6,7])-0.5, colors=plt.rcParams['axes.prop_cycle'].by_key()['color'][0:6])
    
    # misc. details
    cbar = fig.colorbar(cax, ticks=[1,2,3,4,5,6])
    cbar.ax.set_yticklabels(['Africa','Asia','Europe','North America','Oceania','South America'])
    cbar.ax.invert_yaxis()
    plt.tight_layout()
    
    for ax in axs:
        ax.set_ylabel('pc2')
        ax.set_xlabel('pc1')
        ax.set_xlim(lims[0]), ax.set_ylim(lims[1])
        ax.set_aspect('equal')
        
    def _plot_probs(nX, gpc):
        ''' this function plots the probability surface of each class '''
        # predict the probability of each class for nX
        C = gpc.predict_proba(nX)
        # create a plot
        fig, axs = plt.subplots(2,3, figsize=(16, 11), sharex=True, sharey=True)
        # standardise colourmap
        clims = np.min(C.ravel()), np.max(C.ravel())
        # for each class
        for i in range(6):
            # i^th axis
            ax = axs[i//3, i%3]
            
            # plot the training points for that class
            ax.plot(z[np.where(y == i+1),0],z[np.where(y==i+1),1],'wo',label='y')
            
            # plot the probability surface for the current class (in log space)
            cax = ax.pcolor(xg,yg,C[:,i].reshape(50,50), norm=LogNorm(vmin=clims[0], vmax=clims[1]))
            
            # misc. details
            ax.set_xlim(lims[0])
            ax.set_title('p(' + ['Africa','Asia','Europe','North America','Oceania','South America'][i] + ')')
            if i%3 == 0:
                ax.set_ylabel('pc2')
            if i//3 == 1:
                ax.set_xlabel('pc1')
            ax.set_aspect('equal')
        plt.tight_layout()
        # colour bar
        cbar = fig.colorbar(cax, ax=axs.ravel().tolist(), shrink=0.5)
    
    ## this is a bit of trickery to save classifer state outside the demo function
    global plot_probs
    plot_probs = lambda: _plot_probs(nX, gpc) # plot_probs is a function that will evaluate this when called


In [ ]:
%matplotlib inline
plot_classifier()

In [ ]:
%matplotlib inline
plot_probs()

## unsupervised learning

### unsupervised learning

we have $y$ but don't know $X$

we have $X$ but don't know $y$

already covered PCA (latent variable model): find $z, f$ to represent $X$

### clustering

unsupervised classification

$$ z = f(X, \theta)$$
$$ y = \pi(z) $$

how to define $\mathcal{L}$ with no examples of $y$

### k-means
simplest clustering method: sort into fixed number of classes, $k$, based on optimal distances from centre of clusters

In [ ]:
from sklearn.cluster import KMeans

def cluster_kmeans():
    ''' this example will cluster the principle components for the happiness data examples and compare continents to classes'''
    # reinitialise the data 
    data = pd.read_csv('https://wilocw.github.io/files/resource/global/2019.csv')
    data.rename(lambda s: s.replace(' ','_'), axis='columns', inplace=True)

    # standardise the data (this is equiv to do it manually it's just easier this way)
    X = StandardScaler().fit_transform(data.loc[:,features].values)
    
    # we're going to cluster the 2 principle components from earlier in the talk 
    z = PCA(2).fit_transform(X)

    # labels
    labels = ['Africa','Asia','Europe','North America','Oceania','South America']
    # convert label data into integers (1:6)
    y = np.array([{label:i+1 for i,label in enumerate(labels)}[cont] for cont in data.loc[:,'Continent']])
    # plotting limits
    lims = ((-4, 5), (-3, 4))
    # create a mesh to plot 2-D classification surface on
    xg,yg = np.meshgrid(np.linspace(*lims[0],50),np.linspace(*lims[1],50))
    nX = np.vstack((xg.ravel(), yg.ravel())).T

    # create a plot
    fig, axs = plt.subplots(2,3, figsize=(16, 11), sharex=False, sharey=False)
    
   
    
    # kmeans with different number of clusters k
    n_clusters = [2, 4, 6]
    for i, n in enumerate(n_clusters):
        # cluster the data
        kmeans = KMeans(n_clusters=n).fit(z)
        # cluster plotting mesh
        Z = kmeans.predict(nX)
        # plot clustered mesh
        cax = axs[0,i].contourf(xg, yg, Z.reshape(50,50), levels=np.arange(7)-0.5)
        # plot training data (no labels)
        axs[0,i].plot(z[:,0],z[:,1],'o')
        
        # misc. details
        axs[0,i].set_xlim(lims[0])
        axs[0,i].set_ylim(lims[1])
        axs[0,i].set_xlabel('pc1')
        axs[0,i].set_aspect('equal')

        # cluster training data
        zc = kmeans.predict(z)
        width=0.7/6
        for j in range(6):
            # for each class, calculate the proportion of points in each cluster 
            counts = [np.mean(zc[np.where(y == (j+1))] == k) for k in range(n)]
            # then plot it in a bar chat
            axs[1,i].bar(np.arange(n) - (6-j-1)*width, counts, width, label=labels[j])
            
        # misc. details
        axs[1,i].legend()
        axs[1,i].set_xticks([k - 0.35 for k in range(n)])
        axs[1,i].set_xticklabels([str(k+1) for k in range(n)])
        axs[1,i].set_xlabel('cluster')
    plt.tight_layout()

    axs[0,0].set_ylabel('pc2')
    axs[1,0].set_ylabel('proportion')

In [ ]:
%matplotlib inline
cluster_kmeans()

### Gaussian mixture models

clusters now correspond to independent Gaussian distributions each indicating probability for a given $X$

In [ ]:
from sklearn.mixture import BayesianGaussianMixture, GaussianMixture
from matplotlib.patches import Ellipse

def get_ellipse(m, C, col='auto'):
    ''' some utility code to draw an ellipse based on 95% CI of Gaussian '''
    v,w = np.linalg.eigh(C)
    v = 1.96 * np.sqrt(2.) * np.sqrt(v)
    u = w[0] / np.linalg.norm(w[0])
    angle = 180. * np.arctan2(u[1],u[0]) / np.pi
    if col is 'auto':
        ell = Ellipse(m, v[0], v[1], 180. + angle)
    else:
        ell = Ellipse(m, v[0], v[1], 180. + angle, color=col)
            
    return ell


def cluster_gmm():
    ''' this example applies clustering using Gaussian mixture models '''
    # reinitialise the data 
    data = pd.read_csv('https://wilocw.github.io/files/resource/global/2019.csv')
    data.rename(lambda s: s.replace(' ','_'), axis='columns', inplace=True)

    # standardise the data (this is equiv to do it manually it's just easier this way)
    X = StandardScaler().fit_transform(data.loc[:,features].values)
    
    # we're going to cluster the 2 principle components from earlier in the talk 
    z = PCA(2).fit_transform(X)

    # labels
    labels = ['Africa','Asia','Europe','North America','Oceania','South America']
    # convert label data into integers (1:6)
    y = np.array([{label:i+1 for i,label in enumerate(labels)}[cont] for cont in data.loc[:,'Continent']])
    # plotting limits
    lims = ((-4, 5), (-3, 4))

    # create a plot
    fig, axs = plt.subplots(1,3, figsize=(16, 5), sharex=False, sharey=False)
    
    # kmeans with different number of gaussians in mixture
    n_clusters = [2, 4, 6]
    for i, n in enumerate(n_clusters):
        # get colour order (for plotting)
        cols = plt.rcParams['axes.prop_cycle'].by_key()['color'][0:n]
        
        # fit GMM
        gmm = GaussianMixture(n_components=n).fit(z)
        # get means and covariances of each gaussian and hard classify z
        ms, Cs, Z = gmm.means_, gmm.covariances_, gmm.predict(z)
        
        # plot each gaussian in model
        for k in range(n):
            # create ellipse representing 95% CI of current Gaussian
            ell = get_ellipse(ms[k], Cs[k], col=cols[k])
            ell.set_clip_box(axs[i].bbox)
            ell.set_alpha(0.5)
            axs[i].add_artist(ell)
            # plot the points hard classified to the current component
            axs[i].plot(z[Z==k,0],z[Z==k,1],'o', c=cols[k])
        # misc. details
        axs[i].set_xlim(lims[0])
        axs[i].set_ylim(lims[1])
        axs[i].set_xlabel('pc1')
        axs[i].set_aspect('equal')

    plt.tight_layout()
    axs[0].set_ylabel('pc2')

In [ ]:
%matplotlib inline
cluster_gmm()

## key points

### key points

- most machine learning is regression

- regression is just statistics

- machine learning is not _just_ statistics (it's optimisation)

- no free lunch

- learn basic maths prerequisites

- familiarise yourself with basic methods

- rule out the easy things first

### some resources


### mathematics for machine learning (book)


<center><img src="https://wilocw.github.io/files/resource/global/mml-book-cover.jpg"></img>

### [mml-book.github.io](https://mml-book.github.io)

</center>


### machine learning (a probabilistic perspective)

<center><img width=600px src="https://wilocw.github.io/files/resource/global/ml_bookjpg.jpg"></img>

### [probml.github.io/pml-book](https://probml.github.io/pml-book)</center>

### deep learning

<center><img src="https://wilocw.github.io/files/resource/global/deepbook.jpg"></img>

### [deeplearningbook.org](http://deeplearningbook.org)
</center>

### summer schools

<div class='container'>
    
<div class='col' style='text-align: center'>
<img src="https://wilocw.github.io/files/resource/global/gpss.png"></img>

#### [gpss.cc](http://gpss.cc)
    
</div>
<div class='col' style='text-align: center'>
<img src="https://wilocw.github.io/files/resource/global/mlss.png"></img>

</div>
</div>

### [github.com](https://github.com)

<center><img src="https://wilocw.github.io/files/resource/global/github.png"></img></center>

### [paperswithcode.com](https://paperswithcode.com)

<center><img src="https://wilocw.github.io/files/resource/global/pwc.png"></img></center>

### [arxiv.org](arxiv.org)

<div class='container'>
    
<div class='col' style='text-align: center'>
<img src="https://wilocw.github.io/files/resource/global/arxivml.png"></img>
</div>
<div class='col' style='text-align: center'>
<img src="https://wilocw.github.io/files/resource/global/arxivlg.png"></img>
</div>
</div>

## "hang on, i asked about ..."

### graphs

graphs are non-trivial structures; inference has same basic structure

$$ G(X; W) = (V, E) $$
$$ z = f(G(X), W)$$
$$ y = \pi(z)$$

general ideas and problem categories are the same; methods need to be adapted to network, e.g. on embeddings, adjacency matrices

<div class='container'>
    
<div class='col' style='text-align: center'>
<img src="https://wilocw.github.io/files/resource/global/graph_taxonomy.PNG"></img>
</div>
<div class='col' style='text-align: center'>
<img src="https://wilocw.github.io/files/resource/global/graph_review.PNG"></img>
</div>
</div>

<center><img src="https://wilocw.github.io/files/resource/global/pwc_graphs.png"></img>

https://paperswithcode.com/area/graphs
</center>

### 3-D data

what do you mean ?

3-D spatial data ? 3-D volumes, e.g. tomography ?

does a regular method work ? GLMs ? GPs ?

### spatial data

spatial data is multi-format

is it sparsely distributed over a grid ?

can it be modelled as a network ?

spatial statistics can be hard
spatio-temporal statistics is harder

<div class='container'>
    
<div class='col' style='text-align: center'>
<img src="https://wilocw.github.io/files/resource/global/cressie.jpg"></img>
</div>
<div class='col' style='text-align: center'>
<img src="https://wilocw.github.io/files/resource/global/cressiewikle.jpg"></img>
</div>
</div>

### Bayesian model selection

this is about automatically searching the trade-offs discussed

read up on information theory !

information criteria (BIC) used to choose between models

automatic statistician